# Jacobi's Method

In [ ]:
import numpy as np
import numpy.linalg as npla

# New additions!
import scipy
from scipy import sparse

In [ ]:
# Let's do a simple Ax = b problem with a 3x3 matrix A
# Normally, you'd employ a MUCH larger matrix with Jacobi's Method...

A = np.array([[4, -1, -1], [-2, 6, 1], [-1, 1, 7]])
b = np.array([3, 9, -6])
print("A =\n", A, "\n\nb =", b)

# What's the ACTUAL (ideal) solution for x (not iteration, just straight-out solution)??
xideal = npla.solve(A,b)
print("\nIf Ax = b, then x = ", xideal)

### Using Jacobi's Method - the Matrix view:

*What do you need to start off with? See this:*

In [ ]:
# Get dimensions of matrix A:
m, n = A.shape

# Get the diagonals as a vector d:
d = A.diagonal()

# Convert that diagonals vector d into a diagonal MATRIX D:
D = np.diag(d)

print("\nm = ", m, ";","n = ", n, "\n")
print("d =\n", d, "\n")
print("D =\n", D, "\n")

# Create matrix C, which is A WITHOUT the diagonals
C = A - D
print("C =\n", C, "\n")

# Let's make an initial guess: x = 0
x = np.zeros(n)
print ("inital guess for x, i.e. x[0] = ", x)

We KNOW (this is like "cheating" b/c we ran `npla.solve()`) that `x =  [1, 2, -1]`

***So let's improve on the initial guess of x = [0,0,0]:***

In [ ]:
xnew = (b - C @ x) / d
print( "x[1] = ", xnew )

error = npla.norm( A@xnew - b)
print("error:", error)

# residual = xnew - xideal
# error = npla.norm( xnew - xideal )
# relres = npla.norm( xnew - xideal ) / npla.norm( xideal )
# print( "residual:", residual, "\nerror:", error, "\nrelres:", relres )

***Ok - better, but not close enough (relative residual is too high). Do it again!***

In [ ]:
xnew = (b - C @ xnew) / d
print( "x[2] = ", xnew )

error = npla.norm( A@xnew - b)
print("error:", error)


***Ok - AGAIN, it's better, but not close enough (relative residual is too high). Do it again!***

In [ ]:
xnew = (b - C @ xnew) / d
print( "x[3] = ", xnew )

error = npla.norm( A@xnew - b)
print("error:", error)


### Ok - you see where this is going? Better do a loop!

In [ ]:
#Again, start with our initial guess of [0,0,0]:
x = np.zeros(3)

for i in range( 100 ):
    x = (b - C @ x) / d
    error = npla.norm( A@x - b)
    print( "iteration", i + 1, "x:", x, ", error:" ,error )
    if error <= 1e-8:
        break

### We see from the results above that if we (arbitrarily) chose a threshold of 1e-8, that iteration number 19 would get us just below that!!

## ***BUT!!!*** Jacobi's Method does not always converge...! :(

In [ ]:
# Example that does NOT converge using J. Method:

A = np.array([[1,2],[3,4]])
b = np.array([3,7])
print("A:\n", A)
print("\nb = ", b)
x = npla.solve(A, b)
print("\nx (ideal) = ", x)

In [ ]:
# Get d, D, and C:
d = A.diagonal()
D = np.diag(d)
C = A - D

#Start with our initial guess of [0,0]:
x = np.zeros(2)

for i in range( 100 ):
    x = (b - C @ x) / d
    error = npla.norm( A@x - b)
    print( "iteration", i + 1, "x:", x, ", error:" ,error )
    if error <= 1e-8:
        break

## We see from the results above that we NEVER CONVERGE!!

#### We could have avoided this "heartache" by checking the "Spectral Radius" of the Matrix A:

In [ ]:
# Check spectral radius
m = npla.inv(D)@C
evs = npla.eig(m)[0]
print(evs)

if max(evs) < 1:
    print("Spectral radius < 1. Will converge.")
else:
    print("Spectral radius >= 1. Will not converge.")

In [ ]:
# Check it again for our earlier matrix A (that DID converge)
# We'll call it matrix Z here, just to distinguish it from matrix A above:

Z = np.array([[4, -1, -1], [-2, 6, 1], [-1, 1, 7]])
d = Z.diagonal()
D = np.diag(d)
C = Z - D

# Check spectral radius
m = npla.pinv(D)@C
evs = npla.eig(m)[0]
print(evs)

if max(evs) < 1:
    print("Spectral radius < 1. Will converge.")
else:
    print("Spectral radius >= 1. Will not converge.")

### Let's create a function that can do all of this for us!

**Presenting function `Jsolve()`:** \
**It takes in our matrix `A`, vector `b`and gives us the best solution for `x` (plus the `resrel`)** \

It should also have as arguments: a threshold tolerance (default = 1e-8), maximum number of iterations (default = 1000)

# ATTENTION:

**`Jsolve()` employs SPARSE MATRICES so that it's use can be extended to very large, sparse matrices, as well as, more "ordinary" ones.**

This means that BEFORE using it, make sure to convert an np.array() type matrix into a sparse one (how to do that is illustrated all the way below):

In [ ]:
def Jsolve(A, b, tol = 1e-8, max_iters = 1000, callback = None):
    """Solve a linear system Ax = b for x by the Jacobi iterative method.
    Parameters: 
      A: the matrix.
      b: the right-hand side vector.
      tol = 1e-8: the relative residual at which to stop iterating.
      max_iters = 1000: the maximum number of iterations to do. 
      callback = None: a user function to call at every iteration. 
        The callback function has arguments 'x', 'iteration', and 'residual'
    Outputs (in order):
      x: the computed solution
      rel_res: list of relative residual norms at each iteration.
        The number of iterations actually done is len(rel_res) - 1
    """
    # Check the input
    m, n = A.shape
    assert m == n, "matrix must be square"
    bn, = b.shape
    assert bn == n, "rhs vector must be same size as matrix"

    # Split A into diagonal D plus off-diagonal C
    d = A.diagonal()         # diagonal elements of A as a vector
    C = A.copy()             # copy of A ...
    C.setdiag(np.zeros(n))   # ... without the diagonal
    
    # Initial guess: x = 0
    x = np.zeros(n)

    # Vector of relative residuals
    # Relative residual is norm(residual)/norm(b)
    # Intitial residual is b - Ax for x=0, or b
    rel_res = [1.0]
        
    # Call user function if specified
    if callback is not None:
        callback(x = x, iteration = 0, residual = 1)

    # Iterate
    for k in range(1, max_iters+1):
        # New x
        x = (b - C @ x) / d

        # Record relative residual (this can be done instead of error)
        # Remember: rel_res = error / some_relative_reference
        this_rel_res = npla.norm(b - A @ x) / npla.norm(b)
        rel_res.append(this_rel_res)
                
        # Call user function if specified
        if callback is not None:
            callback(x = x, iteration = k, residual = this_rel_res)
                        
        # Stop if within tolerance    
        if this_rel_res <= tol:
            break
            
    return (x, rel_res)

In [ ]:
A = np.array([[4, -1, -1], [-2, 6, 1], [-1, 1, 7]])
b = np.array([3, 9, -6])
print("A:\n", A)
print("\nb:\n", b)
x = npla.solve(A, b)
print("\nIdeal x (so we can compare against it):\n", x)

In [ ]:
#Run it using Jacobi - note, Jsolve() requires A to be a sparse matrix
A = sparse.csr_matrix(A)

print("x: \n", Jsolve(A, b)[0])
print("\nAll iterated residuals: \n", Jsolve(A, b)[1])

# To see just the last residual:
# NOTE: [1] indicates element 1 of the function return, which is a list,
#       [-1] indicates the LAST element in that list.

print("\nLast residual: ", Jsolve(A,b)[1][-1])

In [ ]:
A = np.array([[4, 0, 1, 0], [2, 7, 7, 2], [1, 1, 4, 4], [0, 0, 2, 6]])
b = np.array([1, 2, 3, 4])
print(npla.solve(A,b))

In [ ]:
d = A.diagonal()
D = np.diag(d)
C = A - D

# Check spectral radius
m = npla.pinv(D)@C
evs = npla.eig(m)[0]
print(evs)

if max(evs) < 1:
    print("Spectral radius < 1. Will converge.")
else:
    print("Spectral radius >= 1. Will not converge.")

In [ ]:
#Run it using Jacobi - note, Jsolve() requires A to be a sparse matrix
A = sparse.csr_matrix(A)
solution = Jsolve(A, b, tol = 1e-8)
print("x: \n",solution[0])
print("\nAll iterated residuals: \n", solution[1])
print("length = ", len(solution[1]))
print("\nLast residual: ", solution[1][-1])